<a href="https://colab.research.google.com/github/Kevin091295/FineTune/blob/main/Finetune_Llama3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** 1. Setup Environment**

In [1]:
!pip install transformers datasets peft bitsandbytes trl accelerate pdfplumber torch

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import pdfplumber
from datasets import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

2. Extract Resumes from **PDF**

In [3]:
from google.colab import files
files.upload()
def extract_text_from_pdf(pdf_path):
    resumes = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                resumes.append(text)
    return resumes

# Extract text from your uploaded PDF
resume_texts = extract_text_from_pdf("/content/synthetic_resume_data.pdf")
print(f"Extracted {len(resume_texts)} resumes")
print(resume_texts[0])

Saving synthetic_resume_data.pdf to synthetic_resume_data.pdf
Extracted 10 resumes
Name: Alice Johnson
Email: alice.johnson@example.com
Phone: (555) 123-4567
Education: BS Computer Science, University of Tech, 2019
Work Experience:
- Software Developer, TechCorp, Jan 2020 - Present
- Built scalable web applications using Python and Django
- Optimized database queries, reducing load time by 30%
- Intern, CodeLabs, Summer 2018
- Assisted in debugging software and writing unit tests
Skills: Python, Django, JavaScript, Git, SQL
Name: Bob Patel
Email: bob.patel@example.com
Phone: (555) 234-5678
Education: MS Data Science, Data University, 2021
Work Experience:
- Data Analyst, Insight Analytics, Jul 2021 - Present
- Analyzed trends using SQL and Tableau for 10+ clients
- Developed predictive models with Python
- Research Assistant, Data University, Aug 2020 - Jun 2021
- Conducted statistical modeling for academic papers
Skills: SQL, Tableau, Python, R, Machine Learning
Name: Clara Nguyen


**3. Prepare Dataset**

In [42]:
def format_dataset(resume_texts):
    data = []
    for resume in resume_texts:
        ResumeQuery = "What is email id of Peter Evans?."
        ResumeAnalyzerRole = f"### ResumeAnalyzerRole: You are resume analyzer, who always gives accurate answer. If the question is unrelated, say 'I can only answer resume questions.'\n\n### ResumeQuery: {ResumeQuery}\n\n### ResumeData: 'peter.evans@example.com"
        data.append({"text": ResumeAnalyzerRole})
    return data

# Format the dataset
formatted_data = format_dataset(resume_texts)
print(formatted_data[0])
train_size = int(0.8 * len(formatted_data))  # 80% train, 20% eval
train_data = formatted_data[:train_size]
eval_data = formatted_data[train_size:]
train_dataset = Dataset.from_list(train_data)
print(train_dataset[0])
# eval_dataset = Dataset.from_list(eval_data)

{'text': "### ResumeAnalyzerRole: You are resume analyzer, who always gives accurate answer. If the question is unrelated, say 'I can only answer resume questions.'\n\n### ResumeQuery: What is email id of Peter Evans?.\n\n### ResumeData: 'peter.evans@example.com"}
{'text': "### ResumeAnalyzerRole: You are resume analyzer, who always gives accurate answer. If the question is unrelated, say 'I can only answer resume questions.'\n\n### ResumeQuery: What is email id of Peter Evans?.\n\n### ResumeData: 'peter.evans@example.com"}


**4. Load Model with 4-bit Quantization (QLoRA)**

In [5]:
# Define quantization config for T4 GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load LLaMA 3.1 model (assuming a 1B variant; adjust to available model)
model_name = "meta-llama/Llama-3.2-1B"  # Placeholder; replace with actual 1B model if available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    device_map="auto",
)
model = prepare_model_for_kbit_training(model)
# Set padding token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
! pip install --upgrade trl

**5. Configure QLoRA**

In [12]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.train()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(128256, 2048)
            (layers): ModuleList(
              (0-15): 16 x LlamaDecoderLayer(
                (self_attn): LlamaAttention(
                  (q_proj): lora.Linear(
                    (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.01, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                   

In [13]:
model.print_trainable_parameters()  # Should show >0 trainable params
for name, param in model.named_parameters():
    if "lora" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False



trainable params: 3,407,872 || all params: 1,239,222,272 || trainable%: 0.2750


In [14]:
for name, param in model.named_parameters():
    if "lora" in name:
        print(f"{name}: requires_grad={param.requires_grad}")

base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: requires_grad=True
base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: requires_grad=True
base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: requires_grad=True
base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: requires_grad=True
base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: requires_grad=True
base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: requires_grad=True
base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: requires_grad=True
base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: requires_grad=True
base_model.model.base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight: requires_grad=True
base_model.model.base_model.

In [ ]:
!pip list | grep -E 'transformers|peft|bitsandbytes|trl'

bitsandbytes                       0.45.3
fastrlock                          0.8.3
peft                               0.14.0
sentence-transformers              3.4.1
transformers                       4.48.3
trl                                0.15.2


**6. Training Code**

In [41]:
training_args = TrainingArguments(
    output_dir="./llama-3.1-1b-resume-finetuned",
    per_device_train_batch_size=1,         # Reduced to fit T4 and stabilize gradients
    gradient_accumulation_steps=1,         # Effective batch size = 4, better for small dataset
    num_train_epochs=10,                    # Reduced to prevent overfitting
    learning_rate=2e-4,                    # Lowered for finer updates
    fp16=True,                             # Kept for T4 compatibility
    logging_steps=5,                       # More frequent logging to monitor progress
    save_steps=10,                         # More frequent saves for small dataset
    save_total_limit=2,                    # Kept as is
    optim="adamw_torch",                   # Kept, good choice
    report_to="none",                      # Kept as is
    gradient_checkpointing=True,           # Added to save memory and allow stable training
    warmup_steps=5,                        # Added for gradual LR ramp-up
    max_grad_norm=0.5,                     # Added gradient clipping to prevent explosions
              # Added to evaluate during training
    eval_steps=10,                         # Evaluate every 10 steps
          # Load the best model based on eval loss
    metric_for_best_model="loss",          # Use loss to determine best model
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,

    tokenizer=tokenizer,
    args=training_args,

)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.model.save_pretrained("./llama-3.1-1b-resume-finetuned")
tokenizer.save_pretrained("./llama-3.1-1b-resume-finetuned")

<ipython-input-41-92a007e879e6>:22: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/8 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Step,Training Loss
5,0.129300
10,0.116600
15,0.099800
20,0.071900
25,0.029700
30,0.003300
35,0.000300
40,0.000100
45,0.000100
50,0.000100


('./llama-3.1-1b-resume-finetuned/tokenizer_config.json',
 './llama-3.1-1b-resume-finetuned/special_tokens_map.json',
 './llama-3.1-1b-resume-finetuned/tokenizer.json')

7. Inference Code (Separate)

In [39]:
!pip install transformers datasets peft bitsandbytes trl accelerate pdfplumber torch --quiet

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Define quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the base model and fine-tuned LoRA adapters
base_model_name = "meta-llama/Llama-3.2-1B"
fine_tuned_model_path = "/content/llama-3.1-1b-resume-finetuned"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
fine_tuned_model = PeftModel.from_pretrained(base_model, fine_tuned_model_path)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)
tokenizer.pad_token = tokenizer.eos_token

# Inference function with improved control
def generate_response(prompt, max_new_tokens=50):  # Reduced to limit repetition
    # Check if prompt is resume-related
    resume_keywords = ["resume", "contact", "experience", "education", "skills", "job", "position", "email"]
    is_resume_related = any(keyword in prompt.lower() for keyword in resume_keywords)

    # Match training prompt format exactly
    prompt="work experience of peter evans"
    full_prompt = (
        "### ResumeAnalyzerRole: You are a resume analyzer, who always gives accurate answer. "
        "If the question is unrelated, say 'I can only answer resume questions.'\n\n"
        f"### ResumeQuery: {prompt}\n\n### ResumeData: {formatted_data} "
    )

    inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")
    outputs = fine_tuned_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.1,  # Lowered for less randomness
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,  # Explicit end token
        repetition_penalty=1.2,  # Penalize repetition
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the answer after "ResumeData:"
    if "### ResumeData:" in response:
        response = response.split("### ResumeData:")[1].strip()
    else:
        response = response.strip()

    # Enforce instruction for unrelated prompts
    if not is_resume_related:
        return "I can only answer resume questions."

    return response




In [40]:
# Test inference
test_prompt = "work experience of peter evans?."
print("Resume-related test:")
print(generate_response(test_prompt))


Resume-related test:
[{'text': "### ResumeAnalyzerRole: You are resume analyzer, who always gives accurate answer. If the question is unrelated, say 'I can only answer resume questions.'\n\n### ResumeQuery: What is email id of Peter Evans?.\n\n


In [36]:
test_prompt = "what is contact of Bob Patel?."
print("Resume-related test:")
print(generate_response(test_prompt))


Resume-related test:
[{'text': "### ResumeAnalyzerRole: You are resume analyzer, who always gives accurate answer. If the question is unrelated, say 'I can only answer resume questions.'\n\n### ResumeQuery: What is email id of Peter Evans?.\n\n


8. Check Accuracy